In [1]:
### Library Imports
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")


In [2]:
### Import Stock Data
def get_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    for ticker in tickers:
        data[ticker] = yf.download(ticker, start_date, end_date)['Adj Close']
        # Reset Headers to be in the same row
        data.reset_index(inplace = True)
    
    return data

### Return Stock Data
tickers = ['AAPL']  # Ticker symbol for the S&P 500
start_date = '2024-01-01'
end_date = '2024-08-01'
stock_data = get_data(tickers, start_date, end_date)
print(stock_data.tail())


[*********************100%%**********************]  1 of 1 completed

          Date        AAPL
141 2024-07-25  217.238556
142 2024-07-26  217.708008
143 2024-07-29  217.987686
144 2024-07-30  218.547043
145 2024-07-31  221.823242


In [10]:
### Compute Drawdowns
def compute_drawdowns(stock_data):
    # Calculate Cumulative Maximum (Peak Value)
    stock_data['Peak Val'] = stock_data['AAPL'].cummax()
    # Calculate Drawdown (Trough Value - Peak Value)
    ''' 
    Difference between the cumulative maximum and the current price at each time point
    '''
    stock_data['Drawdowns'] = stock_data['Peak Val'] - stock_data['AAPL']
    # Calculate the Maximum Drawdown = [(Trough Value - Peak Value) / (Peak Value)]
    stock_data['MDD'] = stock_data['Drawdowns'] / stock_data['Peak Val']

    return stock_data

updated_stock_data = compute_drawdowns(stock_data)
print(updated_stock_data.tail())


          Date        AAPL   Drawdown       MDD    Peak Val  Drawdowns
141 2024-07-25  217.238556  17.309967  0.073801  234.548523  17.309967
142 2024-07-26  217.708008  16.840515  0.071800  234.548523  16.840515
143 2024-07-29  217.987686  16.560837  0.070607  234.548523  16.560837
144 2024-07-30  218.547043  16.001480  0.068222  234.548523  16.001480
145 2024-07-31  221.823242  12.725281  0.054254  234.548523  12.725281


In [15]:
### Calculate N Maximum Drawdowns
def max_drawdowns(updated_stock_data, n):
    # Find local minima by comparing shifts
    ''' 
    Must check the following for each data point in the drawdown series:
    Whether the drawdown at the current index is greater than the drawdown at the next index (shift(-1) which shifts the data upwards)
    Whether the drawdown at the current index is also greater than the drawdown at the previous index (shift(1) which shifts the data downwards)

    True: Indicates that the value at that index is a local minimum, meaning it is greater than the values immediately 
    before and after it in the series. This suggests a trough point in a drawdown scenario.
    '''
    updated_stock_data['Local Minima'] = (updated_stock_data['Drawdowns'] > updated_stock_data['Drawdowns'].shift(-1)) & (updated_stock_data['Drawdowns'] > updated_stock_data['Drawdowns'].shift(1))
    #print(updated_stock_data['Local Minima'])
    

    return updated_stock_data

final_df = max_drawdowns(updated_stock_data, n = 5)


0      False
1      False
2      False
3       True
4      False
       ...  
141     True
142    False
143    False
144    False
145    False
Name: Local Minima, Length: 146, dtype: bool


In [ ]:
### Display Graph of Data with Drawdowns

